# Setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install transformers

     |████████████████████████████████| 573kB 8.9MB/s 
     |████████████████████████████████| 890kB 55.6MB/s 
     |████████████████████████████████| 1.0MB 49.6MB/s 
     |████████████████████████████████| 3.7MB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=4eac4d1ed3d8f392ffdbc2dd8e319f5041ecbc6920bacd255ac82308f0e98e26
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
ROOT_DIR = '/content/gdrive/My Drive/2020 Spring/NLU/Project'
DATA_DIR = '/content/gdrive/My Drive/2020 Spring/NLU/Project/echo'
ECHO_DIR = '/content/gdrive/My Drive/2020 Spring/NLU/Project/echo'

# Exploring summaries from BART pipeline

In [0]:
from transformers import pipeline
import requests
import pprint
import time
import os
from os.path import join as pj
import sys
import pandas as pd
import numpy as np
pp = pprint.PrettyPrinter(indent=14)

In [5]:
docs = pd.read_csv(pj(DATA_DIR, 'summaries.csv'))
qaps = pd.read_csv(pj(DATA_DIR, 'qaps.csv'))
## documentation for summarizer: https://huggingface.co/transformers/main_classes/pipelines.html#summarizationpipeline
# summarize with BART
summarizer_bart = pipeline(task='summarization', model="bart-large-cnn")

Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/modelcard.json' to download model card file.
Creating an empty model card.


In [6]:
temp = docs[docs['set']=='train'].head(5)
for i in temp.summary_tokenized.values:
    print(i)

At Madeline Hall , an old mansion-house near Southampton belonging to the wealthy de Versely family , lives an elderly spinster Miss Delmar , the aunt of the earl de Versely and Captain Delmar . Miss Delmar invites Arabella Mason , the daughter of a deceased , well-liked steward to stay with her as a lower-class guest in the house . Captain Delmar is known to visit his aunt at Madeline Hall frequently , accompanied by his valet Ben Keene , who is also a private marine . Captain Delmar eventually suggests that Ben should propose to Arabella , and the two marry in secret , to the frustration of Miss Delmar and Arabella s mother . The captain is able to smooth over the situation with his aunt , even after it is discovered that Arabella was six months pregnant at the time of the marriage . She later gives birth to a boy , who takes the Captain s Christian name and Ben s surname -- the titular Percival Keene . The family moves to Chatham , after Ben is ordered back with his detachment . Ara

In [7]:
for idx, row in temp.iterrows():
    t0 = time.time() # timer
    summary_temp_bart = summarizer_bart(str(row.summary_tokenized), min_length=1, max_length=80) # change min_ and max_length for different output
    print("Summarization took " + str(round((time.time() - t0) / 60, 2)) + " minutes.")
    pp.pprint(summary_temp_bart[0]['summary_text'])

Summarization took 0.59 minutes.
('Percival Keene is the son of Captain Delmar and Arabella Mason. Percival is '
 'a mischievous boy known for his pranks and disregard for manners. He is '
 'taken in as a cabin boy by the captain of his new navy ship, the H.M. '
 'Calliope. He later dyes his skin tan in the appearance of a mulatto.')
Summarization took 0.41 minutes.
('Dr. Samuel Ferguson sets out to travel across the African continent with the '
 'help of a balloon filled with hydrogen. This voyage is meant to link '
 'together the voyages of Sir Richard Burton and John Hanning Speke in East '
 'Africa with those of Heinrich Barth in the regions of the Sahara and Chad.')
Summarization took 0.41 minutes.
('The story begins when a female lovely named Olivia is chased down and '
 'cornered in a marsh , on the edge of the Vilayet Sea. Her pursuer and former '
 'master is a sadistic rogue named Shah Amurath. But before he can lay hands '
 'on her , a figure rises from the reeds . The newcom

# Use GPT-2 for summarization

## Try without fine-tuning first

In [0]:
import torch
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel

In [9]:
docs.head()

,document_id,set,summary,summary_tokenized
0,0025577043f5090cd603c6aea60f26e236195594,test,"Mark Hunter (Slater), a high school student i...","Mark Hunter ( Slater ) , a high school student..."
1,0029bdbe75423337b551e42bb31f9a102785376f,train,"At Madeline Hall, an old mansion-house near S...","At Madeline Hall , an old mansion-house near S..."
2,00936497f5884881f1df23f4834f6739552cee8b,train,"A scholar and explorer, Dr. Samuel Ferguson, ...","A scholar and explorer , Dr. Samuel Ferguson ,..."
3,00950a3641e6a28b04a6fabf6334140e2deaa9fd,train,The story begins when a female lovely named O...,The story begins when a female lovely named Ol...
4,00ee9e01a0e581e0d8cbf7e865a895147c480c5e,train,Chev Chelios (Jason Statham) is a Los Angeles...,Chev Chelios ( Jason Statham ) is a Los Angele...


In [10]:
qaps.head()

,document_id,set,question,answer1,answer2,question_tokenized,answer1_tokenized,answer2_tokenized
0,0025577043f5090cd603c6aea60f26e236195594,test,Who is Mark Hunter?,He is a high school student in Phoenix.,A loner and outsider student with a radio stat...,Who is Mark Hunter ?,He is a high school student in Phoenix .,A loner and outsider student with a radio stat...
1,0025577043f5090cd603c6aea60f26e236195594,test,Where does this radio station take place?,It takes place in Mark's parents basement.,"Phoenix, Arizona",Where does this radio station take place ?,It takes place in Mark s parents basement .,"Phoenix , Arizona"
2,0025577043f5090cd603c6aea60f26e236195594,test,Why do more students tune into Mark's show?,Mark talks about what goes on at school and in...,Because he has a thing to say about what is ha...,Why do more students tune into Mark s show ?,Mark talks about what goes on at school and in...,Because he has a thing to say about what is ha...
3,0025577043f5090cd603c6aea60f26e236195594,test,Who commits suicide?,Malcolm.,Malcolm.,Who commits suicide ?,Malcolm .,Malcolm .
4,0025577043f5090cd603c6aea60f26e236195594,test,What does Paige jam into her microwave?,She jams her medals and accolades.,Her award medals,What does Paige jam into her microwave ?,She jams her medals and accolades .,Her award medals


In [11]:
qaps.set.value_counts()

train    32747
test     10557
valid     3461
Name: set, dtype: int64

In [12]:
docs.set.value_counts()

train    1102
test      355
valid     115
Name: set, dtype: int64

In [0]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained("gpt2")

indexed_texts = []
# Encode a text inputs
# Convert indexed tokens in a PyTorch tensor
for row in temp.summary_tokenized.values:
    summary_idx = tokenizer.encode(" Summary:")
    text_idx = tokenizer.encode(row, max_length=1022)
    indexed_texts.append(torch.tensor(text_idx+summary_idx))


In [41]:
model.eval()
#model.to('cuda')
tokens_tensor = tokens_tensor.to('cuda')

for row in indexed_texts:
    tokens_tensor = row.unsqueeze(0).to('cuda')
    predicted = model.generate(tokens_tensor, max_length=10, min_length=5, temperature=0.7, do_sample=True)
    print(f"Input:  {tokenizer.decode(row)}")
    print(f"Output: {tokenizer.decode(predicted[0])}")
    # with torch.no_grad():
    #     outputs = model(tokens_tensor)
    #     predictions = outputs[0]

    # # get the predicted next sub-word (in our case, the word 'man')
    # predicted_index = torch.argmax(predictions[0, -1, :]).item()
    # predicted_text = tokenizer.decode([predicted_index])

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Input:  At Madeline Hall, an old mansion-house near Southampton belonging to the wealthy de Versely family, lives an elderly spinster Miss Delmar, the aunt of the earl de Versely and Captain Delmar. Miss Delmar invites Arabella Mason, the daughter of a deceased, well-liked steward to stay with her as a lower-class guest in the house. Captain Delmar is known to visit his aunt at Madeline Hall frequently, accompanied by his valet Ben Keene, who is also a private marine. Captain Delmar eventually suggests that Ben should propose to Arabella, and the two marry in secret, to the frustration of Miss Delmar and Arabella s mother. The captain is able to smooth over the situation with his aunt, even after it is discovered that Arabella was six months pregnant at the time of the marriage. She later gives birth to a boy, who takes the Captain s Christian name and Ben s surname -- the titular Percival Keene. The family moves to Chatham, after Ben is ordered back with his detachment. Arabella opens

## Fine-Tune GPT-2 for summarization